Перебирает мероприятия на TimePad.
Команда TimePad обновила api, и сервис вцелом.
get запросы указаные не исполняются 
код несет исключительно информационный характер как демонстрация методов BeautifulSoup.
В комерческих целях не использовался.

In [ ]:
Запускалось на windows использовалась библиотека concurrent
На линуксе удобно запускать в многопроцессорном режиме
    from multiprocessing import Pool
    number_processes=3
    with Pool(number_processes) as p:
        p.map(make_all2,all_links)


In [ ]:
import csv
from bs4 import BeautifulSoup
import requests, re, pandas as pd
from datetime import datetime



In [ ]:
def get_page_data(html):
    soup = BeautifulSoup(html, 'lxml')
    k_back=soup.find('span', class_='ep3-pagesummary__time-short').text.strip()
    temp=[]
    if  not k_back.find('назад')+1:  #нет слова "назад" в дате - значит мероприятие еще не произошло
        #город            #онлайн - без города
        k=soup.find('span', class_='ep3-pagesummary__place-city')
        temp.append(try_text_strip(k))

        #адрес __time-begin">
        k=soup.find('div', class_='ep3-pagesummary__place-adress').find('span')
        temp.append(try_text_strip(k))
        
        #date and время   __time-begin"> \xa0
        k=soup.find('div', class_='ep3-pagesummary__time-begin').find('span').text.strip()
        date=' '.join(re.findall(r'(\d{1,2}\s[А-я]+)',k))
        time2=' - '.join(re.findall(r'(\d{1,2}:\d{2})',k))
        temp.append(date)
        temp.append(time2)
        
        #5время относительное  __time-short"> \xa0
        temp.append(re.sub("\xa0", " ",k_back))
        
        #краткое описание   shortdesc
        k=soup.find('p', class_='ep3-pagesummary__shortdesc') #.text.strip()
        temp.append(try_text_strip(k))

        #организатор orgname
        k=soup.find('div', class_='ep3-pagesummary__orgname').find('span') #.text.strip()
        temp.append(try_text_strip(k))
        
        #номер оргов контакты и имя
            
    return temp 

def try_text_strip(k):
        try:
            return(re.sub("\xa0", " ",k.text.strip()) )
        except:
            return ''

In [ ]:
import sys
def get_html(url,params={}):
    try:
        res=requests.get(url, params)    
    except requests.exceptions.TooManyRedirects:
        print('TooManyRedirects', i, len(data_pages),datetime.now() - timerStart)
        return []
    except:
        print (sys.exc_info()[0], i, 'err')
        return []
    if res.status_code==200:
        #print(res.status_code,url, len(res.content),sep='\n')
        #return get_data(re.sub(" +", " ", res.text))
        return re.sub(" +", " ", res.text)
    else:
        print(res.status_code, url)
        return []

def write_excel(data):    
    df=pd.DataFrame(data)
    df.to_excel('T2.xlsx')
  

In [ ]:
import concurrent.futures

def make_all_2(url):  #miltipricessig
    global data_pages
    html = get_html(url)     
    t = get_page_data(html)    
    t.append(url)
    data_pages.append(t)
    return t

timerStart = datetime.now()
data_pages=[]

api_url='http://timepad.ru/event/'
k=1025381
all_links=[api_url+str(k+i) for i in range(150)]

number_processes=30
with concurrent.futures.ThreadPoolExecutor(number_processes) as executor:        
    #futures2=executor.map(make_all_2,all_links)    #1 варик
    futures2 = [executor.submit(make_all_2,k) for k in all_links]  #2ой варик
    concurrent.futures.wait(futures2)    
 
write_excel(data_pages)

print ( datetime.now()- timerStart) 
